### Get data selected for leanring variability project

In [3]:
"""
IMPORTS
"""
import numpy as np
from oneibl.onelight import ONE
import numpy.random as npr
import pandas as pd
import json
from collections import defaultdict
import wget
from zipfile import ZipFile
import os
import pickle
from preprocessing_utils_ines import get_animal_name, load_animal_list

npr.seed(65)

In [6]:
"""
GET ANIMALS OF INTEREST
"""
# Animals in this path were filtered based on having finished training, belonging to the brainwidemap project
# TODO: should add here the code that generates the data I'm using here

trials_dir = '/home/ines/repositories/learning_variability/DATA/'
all_trials = pd.read_csv(trials_dir + "learning_private_one.csv")  
animal_list = all_trials['subject_nickname'].unique()
data_dir = '/home/ines/repositories/learning_variability/DATA/GLMHMM/'
os.chdir(data_dir)
np.savez('animal_list.npz', animal_list)

/home/ines/miniconda3/envs/glmhmm/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3258: DtypeWarning: Columns (23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
data_dir = '/home/ines/repositories/learning_variability/DATA/GLMHMM/'
os.chdir(data_dir)
animal_list = load_animal_list('animal_list.npz')

In [13]:
"""
GET eids of interest
"""
# change directory so that ONE searches in correct directory:
one_local_rep_path = '/home/ines/Downloads/ONE/alyx.internationalbrainlab.org/'
os.chdir(one_local_rep_path)
one = ONE()

# Get all eids in the local database
eids = one.search(['_ibl_trials.*'])
assert len(eids) > 0, "ONE search is in incorrect directory"

# Filter eids based on animal_list
animal_eid_dict = defaultdict(list)

for eid in eids:
    animal = get_animal_name(eid)
    raw_session_id = eid.split('Subjects/')[1]
    date = raw_session_id[-14:-4]
    session_no = raw_session_id[-1:]

    animal_session = all_trials.loc[(all_trials['session_date']==date) &
                            (all_trials['subject_nickname']==animal) &
                            (all_trials['session_number']==int(session_no))]
    if len(animal_session) > 0:
        animal_eid_dict[animal].append(eid)
        
os.chdir(data_dir)
json = json.dumps(animal_eid_dict)
f = open("animal_eid_dict.json",  "w")
f.write(json)
f.close()
